# Train a Chess LLM on xLAN Datasets/Validate Model

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from src.train import ChessTrainer

## Dataset

- use datasets in ./data/training to train your model
- use DownloadUpload.ipynb to Download a dataset from Hugging Face
- use DataPreProcessing.ipynb to create your own dataset

### CUDA
Check if CUDA is available.

In [ ]:
print(torch.cuda.is_available())

### Config

### Datasets

In [ ]:
# xLAN

ds_19k = "./data/training/09_2023/xlan/standard_rated_19k.xlan.tok"
ds_71k = "./data/training/09_2023/xlan/standard_rated_71k.xlan.tok"
ds_350k = "./data/training/09_2023/xlan/standard_rated_350k.xlan.tok"

# xLAN+
ds_19k_plus = "./data/training/09_2023/xlanplus/standard_rated_19k.xlanplus.tok"
ds_71k_plus = "./data/training/09_2023/xlanplus/standard_rated_71k.xlanplus.tok"
ds_350k_plus = "./data/training/09_2023/xlanplus/standard_rated_350k.xlanplus.tok"
ds_1M_plus = "./data/training/09_2023/standard_rated_1M.xlanplus.tok"

# finetune low elo
ds_10k_low_elo = "./data/training/03_2024/0_1000_elo/standard_rated_low_elo_10k_bos.tok"
ds_30k_low_elo = "./data/training/03_2024/0_1000_elo/standard_rated_low_elo_30k_bos.tok"
ds_98k_low_elo = "./data/training/03_2024/0_1000_elo/standard_rated_low_elo_98k_bos.tok"

# finetune high elo
ds_10k_high_elo = (
    "./data/training/03_2024/2000_3500_elo/standard_rated_high_elo_10k_bos.tok"
)
ds_30k_high_elo = (
    "./data/training/03_2024/2000_3500_elo/standard_rated_high_elo_30k_bos.tok"
)
ds_98k_high_elo = (
    "./data/training/03_2024/2000_3500_elo/standard_rated_high_elo_98k_bos.tok"
)

In [ ]:
dataset = ds_30k_high_elo

## HYPERPARAMETERS
BATCH_SIZE = 16  # use the largest batch size that fits on your GPU
SAVE_STEPS = 2000  # how often to save a checkpoint
LOGGING_STEPS = 50  # how often to validate model and publish it to Weights & Biases
EPOCHS = 4  # how many epochs to train for - how many times to go through the dataset
LEARNING_RATE = 0.0001  # learning rate - how fast the model should learn
SKIP_VALIDATION = False  # skip validation and only save model checkpoints
WEIGHTS_AND_BIASES_ENABLED = True  # enable logging to Weights & Biases
USE_FP16 = False  # enable mixed precision training (GPU only)
XLANPLUS_ENABLED = True  # use xLanPlus tokenizer

# PEFT_BASE_MODEL = "Leon-LLM/Leon-Chess-350k-BOS"  # base model to be loaded (from hugging face) for fine-tuning
PEFT_BASE_MODEL = "Leon-LLM/V63_GPT2_350k_4E_xLANplus_RIGHT_PAD"  # base model to be loaded (from hugging face) for fine-tuning
# PEFT_BASE_MODEL = "./Leon-LLM-Models/V45_GPT2_19k_20E_xLANplus/checkpoint-24000"  # base model to be loaded (from disk) for fine-tuning

## NAMING
MODEL_TYPE = "GPT2"  # choose between "GPT2" or "Mamba"
VERSION_NUMBER = "V69"  # chronological version increment
DATASET_SIZE = "30k"  # Options: 19k, 71k, 350k, 1M

NOTATION = "xLANplus" if XLANPLUS_ENABLED else "xLAN"

## CONFIG FOR FINE-TUNING
R = 64  # 8~26min, 16~26min, 32~26min
LORA_ALPHA = 32
LORA_DROPOUT = 0.1

peft_config = LoraConfig(  # https://huggingface.co/docs/peft/v0.10.0/en/package_reference/lora#peft.LoraConfig
    task_type="CAUSAL_LM",  # This does not need to be changed for our use case
    inference_mode=False,  # don't change this for training, only later for inference
    r=R,  # lower means faster training, but might underfit because of less complexity (experiments don't show that training time increases, which is rather weird)
    lora_alpha=LORA_ALPHA,  # scaling factor that adjusts the magnitude of the combined result (balances the pretrained model’s knowledge and the new task-specific adaptation)
    lora_dropout=LORA_DROPOUT,
    # use_rslora=True, # might work better (not tried yet)
)

peft_model = get_peft_model(
    AutoModelForCausalLM.from_pretrained(PEFT_BASE_MODEL), peft_config
)

## MODEL NAME
model_name = (
    f"{VERSION_NUMBER}_LoRA_V63_GPT2-350k-Plus_{DATASET_SIZE}_high_elo_{EPOCHS}E_r{R}"
    if PEFT_BASE_MODEL
    else f"{VERSION_NUMBER}_{MODEL_TYPE}_{DATASET_SIZE}_{EPOCHS}E_{NOTATION}"
)

## SAVING MODEL
output_dir = f"./Leon-LLM-Models/{model_name}/"

print(model_name)

### Train

In [ ]:
trainer = ChessTrainer(
    model_type=MODEL_TYPE,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    epochs=EPOCHS,
    input_file=dataset,
    output_dir=output_dir,
    save_steps=SAVE_STEPS,
    logging_steps=LOGGING_STEPS,
    skip_validation=SKIP_VALIDATION,
    weight_and_biases=WEIGHTS_AND_BIASES_ENABLED,
    use_FP16=USE_FP16,
    notation="xLANplus" if XLANPLUS_ENABLED else "xLAN",
    peft=peft_model,
)


trainer.train()

# Push Model to Hugging Face

In [ ]:
from huggingface_hub import notebook_login

# notebook_login()
peft_model.push_to_hub(model_name, organization="Leon-LLM")

# Load Model from Disk (fine-tuned LoRA model)

In [ ]:
# https://huggingface.co/docs/transformers/main/en/peft
model_dir = "./Leon-LLM-Models/V45_GPT2_19k_20E_xLANplus/checkpoint-24000"
peft_model_id = "./Leon-LLM-Models/V55_V45_GPT2_19k_20E_xLANplus_19k_1E_r128/V55_V45_GPT2_19k_20E_xLANplus_19k_1E_r128"
loaded_model = AutoModelForCausalLM.from_pretrained(model_dir)
loaded_model.load_adapter(peft_model_id)

In [ ]:
from src.generate_prediction import generate_prediction

loaded_model.inference_mode = True
loaded_model.eval()
input = "Pd2d4 Pd7d5 Pc2c4 Pc7c6"
loaded_model.to("cuda" if torch.cuda.is_available() else "cpu")
generate_prediction(
    input=input,
    num_tokens_to_generate=3,
    model=loaded_model,
    notation="xLAN",
)[0]

# Inference (for fine-tuned LoRA model)

In [ ]:
# load model from hugging face

from src.generate_prediction import generate_prediction

peft_model = get_peft_model(
    AutoModelForCausalLM.from_pretrained(PEFT_BASE_MODEL), peft_config
)
peft_model.inference_mode = True
peft_model.eval()
input = "Pd2d4 Pd7d5 Pc2c4 Pc7c6"
peft_model.to("cuda" if torch.cuda.is_available() else "cpu")
generate_prediction(
    input=input,
    num_tokens_to_generate=3,
    model=peft_model,
    notation="xLAN",
)[0]